In [1]:
from typing import List, Any
import pandas as pd
import os

In [2]:
TEMPLATE = "INSERT INTO TABLE_NAME (COLUMNS) VALUES\nDATA;"

In [3]:
def transform_item(i: Any):
    if type(i) == str:
        return f"'{i}'"
    elif i == None:
        return 'NULL'
    return str(i)

def transform_list(l: List):
    return list(map(transform_item, l))

def transform_list_to_str(l: List):
    tmp = map(lambda l_: ', '.join(l_), l)
    tmp = map(lambda x: f'({x})', tmp)
    return ',\n'.join(tmp)

# print(transform_list_to_str([["'string'", '2', '3.4'], ["'string2'", '3', '4.2']]))

In [4]:
def generate_insert_tuples(table_name: str, columns: List, values: List[List[Any]]):
    insert = TEMPLATE.replace('TABLE_NAME', table_name)
    
    if len(columns) <= 0:
        insert = insert.replace('(COLUMNS) ', '')
    else:
        insert = insert.replace('COLUMNS', ', '.join(columns))
        
    values_ = map(transform_list, values)
    values_ = transform_list_to_str(values_)
    
    return insert.replace('DATA', values_)

print(generate_insert_tuples('ANIME', ['teste', 'a'], [[1, 'b'], [2, 'c']]))

INSERT INTO ANIME (teste, a) VALUES
(1, 'b'),
(2, 'c');


In [5]:
def get_unique_values(df: 'pd.DataFrame', column):
    values = set()
    for i in df[column]:
        for name in i.split(', '):
            values.add(name)
    return [[i] for i in list(values)]

In [6]:
def id_gen():
    i = 0
    while True:
        i += 1
        yield i

def get_ids(data: List[List[Any]]):
    id_ = id_gen()
    return [[i[0], next(id_)] for i in data]

In [7]:
def create_pairs(df, data, column):
    data_dict = dict(data)
    list_ = list()
    pair = zip(df['MAL_ID'], df[column])
    for anime_id, d in pair:
        dl = d.split(', ')
        for i in dl:
            list_.append([anime_id, data_dict[i]])
    return list_

In [8]:
TRANSLATE = {
    'Spring': 'primavera',
    'Winter': 'inverno',
    'Summer': 'verao',
    'Fall': 'outono'
}
def season_split(x):
    s = x.split()
    if len(s) < 2 or s[0] not in ('Spring', 'Summer', 'Winter', 'Fall'):
        return None
    else:
        return TRANSLATE[s[0]]
    
def year_split(x):
    y = x.split()
    if len(y) < 2:
        return None
    else:
        return y[1]

In [9]:
os.listdir('anime')

['html folder',
 'anime.csv',
 'rating_complete.csv',
 'anime_with_synopsis.csv',
 'watching_status.csv',
 'animelist.csv']

In [10]:
df = pd.read_csv('anime/anime.csv')

In [11]:
df['Producers'] = df['Producers'].apply(lambda s: s.replace("'", ' '))
df['Licensors'] = df['Licensors'].apply(lambda s: s.replace("'", ' '))
df['Name'] = df['Name'].apply(lambda n: n.replace("'", " "))
df['Studios'] = df['Studios'].apply(lambda s: s.replace("'", ' '))

In [12]:
genres = get_ids(get_unique_values(df, 'Genres'))
producers = get_ids(get_unique_values(df, 'Producers'))
licensors = get_ids(get_unique_values(df, 'Licensors'))
studios = get_ids(get_unique_values(df, 'Studios'))
anime_genres = create_pairs(df, genres, 'Genres')
anime_producers = create_pairs(df, producers, 'Producers')
anime_licensors = create_pairs(df, licensors, 'Licensors')
anime_studios = create_pairs(df, studios, 'Studios')

data_ = [
    ('GENERO', ['nome', 'id'], genres), 
    ('PRODUTOR', ['nome', 'id'], producers),
    ('LICENCIADOR', ['nome', 'id'], licensors),
    ('ESTUDIO', ['nome', 'id'], studios),
    ('participa', ['anime', 'genero'], anime_genres),
    ('produz', ['anime', 'produtor'], anime_producers), 
    ('licencia', ['anime', 'licenciador'], anime_licensors), 
    ('anima', ['anime', 'estudio'], anime_studios)]

In [13]:
dfa = df
dfa.drop(['Genres', 'Producers', 'Licensors', 'Studios'], axis=1, inplace=True)
dfa.drop(['English name', 'Japanese name'], axis=1, inplace=True)
dfa.drop(['Aired'], axis=1, inplace=True)

In [14]:
dfa['year'] = dfa['Premiered'].apply(year_split)
dfa['season'] = dfa['Premiered'].apply(season_split)
dfa.drop(['Premiered'], axis=1, inplace=True)

In [15]:
dfa['season'].unique()

array(['primavera', None, 'verao', 'outono', 'inverno'], dtype=object)

In [16]:
dfa['Score'] = dfa['Score'].replace(['Unknown'], 0)
dfa['Type'] = dfa['Type'].replace(['Unknown'], None)
dfa['Episodes'] = dfa['Episodes'].replace(['Unknown'], 0)
dfa['Source'] = dfa['Source'].replace(['Unknown'], 'Other')
dfa['Ranked'] = dfa['Ranked'].replace(['Unknown'], 0)

In [17]:
dfa.columns

Index(['MAL_ID', 'Name', 'Score', 'Type', 'Episodes', 'Source', 'Duration',
       'Rating', 'Ranked', 'Popularity', 'Members', 'Favorites', 'Watching',
       'Completed', 'On-Hold', 'Dropped', 'Plan to Watch', 'Score-10',
       'Score-9', 'Score-8', 'Score-7', 'Score-6', 'Score-5', 'Score-4',
       'Score-3', 'Score-2', 'Score-1', 'year', 'season'],
      dtype='object')

In [18]:
for i in range(1, 11):
    name = f'Score-{i}'
    dfa[name] = dfa[name].replace(['Unknown'], 0)

In [19]:
dfa.drop(['Duration'], axis=1, inplace=True)
dfa.drop(['Rating'], axis=1, inplace=True)
dfa.drop(['Watching', 'Completed', 'On-Hold', 'Dropped', 'Plan to Watch'], axis=1, inplace=True)

In [20]:
table_columns = ['id', 'nome', 'nota', 'tipo', 'num_episodios', 'material_original',
                '`rank`', 'popularidade', 'num_membros', 'num_favoritos', 'num_nota10',
                'num_nota9', 'num_nota8', 'num_nota7', 'num_nota6', 'num_nota5',
                'num_nota4', 'num_nota3', 'num_nota2', 'num_nota1', 'ano', 'estacao']

In [21]:
columns = list(dfa.columns)
animes_ = zip(*[dfa[i] for i in columns])

In [22]:
data_ = [('ANIME', table_columns, animes_), *data_]

In [23]:
ws = pd.read_csv('anime/watching_status.csv')
l = zip(ws['status'], ws[' description'])
# acompanhamento
watching_status = [[i, j] for i, j in l]

data_ = [('ACOMPANHAMENTO', ['id', 'description'], watching_status), *data_]

In [24]:
users = set()
    
with open('anime/animelist.csv', 'r') as fd:
    fd.readline()
    for i in fd:
        users.add(i.split(',')[0])

In [25]:
users_ = [[i] for i in users]

In [26]:
data_ = [('USUARIO', ['id'], users_), *data_]

In [27]:
HEADER = '''-- --------  << Data Mining >>  ----------
--
--                    SCRIPT DE INSERCAO (DML)
--
-- Data Criacao ...........: 20/09/2022
-- Autor(es) ..............: João Guedes, Luiz Henrique e Wagner Cunha
-- Banco de Dados .........: MySQL 8.0
-- Base de Dados (nome) ...: anime_rec
--
--
-- PROJETO => 01 Base de Dados
--         => 12 Tabelas
--         => 01 Visao
--
-- ---------------------------------------------------------'''

with open('popula.sql', 'w+') as fd:
    fd.write(HEADER)
    fd.write('\n\n')

In [28]:
with open('popula.sql', 'a+') as fd:
    for i in data_:
        fd.write(generate_insert_tuples(*i))
        fd.write('\n\n')

A célula abaixo adiciona ao arquivo popula apenas um subconjunto (500 mil tuplas) da tabela 'assiste'. Como a tabela inteira tem um volume muito grande, recomenda-se utilizar isso para ser mais rápido. **Não execute as células abaixo se executar esta**.

In [128]:
out = open('popula.sql', 'a+')
out.write('INSERT INTO assiste (usuario, anime, nota, acompanhamento, num_episodios_assistidos) VALUES\n')
limit = 0
with open('anime/animelist.csv', 'r') as fd:
    fd.readline()
    last = f'({fd.readline().strip()})'
    for i in fd:
        if limit > 500000:
            break
        out.write(f'{last},\n')
        last = f'({i.strip()})'
        limit += 1
    out.write(f'{last};\n\n')
out.close()

As últimas células criam INSERTs em batch de 500 mil tuplas. Isso é feito pois existe um limite no mysql da quantidade de dados que podem ser enviados para um único comando.

In [29]:
def write_newlines(fd): fd.write(';\n\n')

In [30]:
acomp = [1, 2, 3, 4, 6]
def is_in_acompanhamento(x):
    return int(x.split(',')[3]) in acomp

In [31]:
out = open('popula.sql', 'a+')
counter = 0
limit = 500000
with open('anime/animelist.csv', 'r') as fd:
    fd.readline() # reads the head
    for line in fd:
        line_ = f'({line.strip()})'
        if not is_in_acompanhamento(line_):
#             print(f'line_error: {line_}')
            continue
        if counter == 0:
            out.write('INSERT INTO assiste (usuario, anime, nota, acompanhamento, num_episodios_assistidos) VALUES\n')
            out.write(line_)
        else:
            out.write(f',{line_}')
        counter += 1;
        if counter >= limit:
            counter = 0
            write_newlines(out)
out.close()